In [60]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from matplotlib import font_manager, rc
font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
rc('font', family=font_name)

import warnings
warnings.filterwarnings('ignore')

In [61]:
# covide-19 글로벌 kaggle 데이터
kag = pd.read_csv('./data/corona-virus-report/covid_19_clean_complete.csv')
kag['Date'] = pd.to_datetime(kag['Date'])

In [62]:
kag.drop(['Province/State', 'Lat', 'Long'], axis=1, inplace=True)

In [63]:
kag.tail()

,Country/Region,Date,Confirmed,Deaths,Recovered
27715,Western Sahara,2020-05-05,6,0,5
27716,Sao Tome and Principe,2020-05-05,174,3,4
27717,Yemen,2020-05-05,22,4,0
27718,Comoros,2020-05-05,3,0,0
27719,Tajikistan,2020-05-05,293,5,0


In [64]:
pivoted_kag = kag.pivot_table(index = 'Date', columns = 'Country/Region', values='Confirmed', aggfunc='sum')

# 주요 국가만 추출해서 사용한다
주요 국가는 질병관리본부 브리핑에 언급되는 나라들로 구성
유럽 국가
영국, 프랑스, 독일, 스페인, 이탈리아
아시아
중국, 대만, 태국, 베트남, 일본, 우즈베키스탄, 카자흐스탄, 파키스탄, 인도, 싱가포르, 아랍에미리트

In [65]:
euro = ['United Kingdom', 'France', 'Germany', 'Spain','Italy']
china = ['China']
asia = ['Japan', 'Kazakhstan', 'Taiwan*', 'Uzbekistan', 'Thailand', 'Vietnam', 'India', 'Pakistan', 'United Arab Emirates', 'Singapore']
north_america = ['US', "Canada"]
#oceania = ['New Zealand', 'Australia']

In [66]:
#new_glob1 = pivoted_kag[euro+china+asia+america]

In [67]:
new_glob1 = pd.DataFrame()

In [68]:
new_glob1['EURO'] = pivoted_kag[euro].sum(axis=1)
new_glob1['CHINA'] = pivoted_kag[china].sum(axis=1)
new_glob1['ASIA'] = pivoted_kag[asia].sum(axis=1)
new_glob1['N_AMERICA'] = pivoted_kag[north_america].sum(axis=1)
new_glob1['NZ'] = pivoted_kag['New Zealand']

In [69]:
new_glob1.head()

,EURO,CHINA,ASIA,N_AMERICA,NZ
Date,,,,,
2020-01-22,0,548,5,1,0
2020-01-23,0,643,9,1,0
2020-01-24,2,920,15,2,0
2020-01-25,3,1406,17,2,0
2020-01-26,3,2075,22,6,0


# 1. 글로벌 확진자 데이터

## 국가 추출
- 주요 국가만 추출해서 사용한다
- 주요 국가는 질병관리본부 브리핑에 언급되는 나라들로 구성

#### 유럽 국가
- 영국, 프랑스, 독일, 스페인, 이탈리아

#### 아시아
- 중국, 대만, 태국, 베트남, 일본, 우즈베키스탄, 카자흐스탄, 파키스탄, 인도, 싱가포르, 아랍에미리트

#### 미주
- 미국

# 2. 로밍데이터

In [70]:
# 로밍데이터
roam1 = pd.read_csv('./data/corona_contest_data_0406/2. Roaming_data.csv')
roam2 = pd.read_csv('./data/corona_contest_data_0429/2. Roaming_data.csv') 
roam3 = pd.read_csv('./data/corona_contest_data_0506/2. Roaming_data.csv') 

In [71]:
roam = pd.concat([roam1, roam2, roam3])
roam.head()

,return,iso,arrival,departure,count
0,20200131,ad,20200130,20200130,3
1,20200131,ae,20191231,20200125,3
2,20200131,ae,20191231,20200131,3
3,20200131,ae,20200102,20200102,3
4,20200131,ae,20200104,20200131,3


In [72]:
print(roam1.shape)
print(roam2.shape)
print(roam.shape)

(212325, 5)
(5480, 5)
(219025, 5)


In [73]:
# 날짜 변수로 변경
roam['return'] = pd.to_datetime(roam['return'], format='%Y%m%d')
roam['arrival'] = pd.to_datetime(roam['arrival'], format='%Y%m%d')
roam['departure'] = pd.to_datetime(roam['departure'], format='%Y%m%d')

# 파생변수 생성
# arrival과 return의 차를 내서 해외에 체류한 일자수를 계산함
roam['stay'] = roam['return'] - roam['arrival']
roam.rename(columns={'return':'Date'}, inplace=True)

In [74]:
euro_dict = {'gb':'영국', 'fr':'프랑스', 'de':'독일', 'es':'스페인', 'it':'이탈리아'}

In [75]:
china_dict = {'cn':'중국', 'hk':'홍콩', 'mo':'마카오'}
asia_dict = {'tw':'대만', 'th':'태국', 'vn':'베트남', 'jp':'일본', 'uz':'우즈베키스탄', 'kz':'카자흐스탄', 'pk':'파키스탄', 'in':'인도', 'sg':'싱가포르', 'ae':'아랍에미리트'}

In [76]:
n_america_dict = {'us':'미국', 'ca':'캐나다'}
#oceania_dict = {'nz':'뉴질랜드', 'au':'호주'}

In [77]:
pivoted_roam = roam.pivot_table(index='Date', 
                 columns='iso',
                 values='count',
                 aggfunc='sum')

In [78]:
pivoted_roam.fillna(0,inplace=True)

In [79]:
new_roam = pd.DataFrame()

In [80]:
new_roam['EURO'] = pivoted_roam[list(euro_dict.keys())].sum(axis=1)
new_roam['CHINA'] = pivoted_roam[list(china_dict.keys())].sum(axis=1)
new_roam['ASIA'] = pivoted_roam[list(asia_dict.keys())].sum(axis=1)
new_roam['N_AMERICA'] = pivoted_roam[list(n_america_dict.keys())].sum(axis=1)
new_roam['NZ'] = pivoted_roam['nz']

In [81]:
new_roam.reset_index(inplace=True)

# Merge Datesets

In [82]:
df = pd.merge(new_glob1, new_roam, on='Date', suffixes=('_conf', '_roam'),)

In [83]:
df.tail()

,Date,EURO_conf,CHINA_conf,ASIA_conf,N_AMERICA_conf,NZ_conf,EURO_roam,CHINA_roam,ASIA_roam,N_AMERICA_roam,NZ_roam
100,2020-05-01,930930,83959,109157,1159804,1485,24.0,230.0,247.0,126.0,0.0
101,2020-05-02,942895,83959,114163,1190465,1487,77.0,90.0,254.0,174.0,0.0
102,2020-05-03,950614,83964,119579,1218544,1487,48.0,130.0,248.0,104.0,0.0
103,2020-05-04,957516,83966,125898,1242332,1486,51.0,94.0,238.0,95.0,0.0
104,2020-05-05,966279,83968,131413,1267566,1488,18.0,142.0,278.0,129.0,0.0


## 한국 해외유입확진자 데이터 로드

In [84]:
ov = pd.read_csv('./data/oversea_inflow.csv')
# 불필요 row 삭제
ov

,date,oversea_inflow,일일_확진자,중국,중국_외_아시아,유럽,미주,아프리카,호주,검역단계,지역사회,내국인,외국인
0,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-01-21,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-01-22,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-01-23,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-01-24,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2020-01-25,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2020-01-26,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2020-01-27,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2020-01-28,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2020-01-29,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [85]:
ov.drop([0,108], axis=0, inplace=True)

In [86]:
ov.rename(columns={'date':'Date', 'oversea_inflow':'TARGET'}, inplace=True)

In [87]:
ov['Date'] = pd.to_datetime(ov['Date'])

In [88]:
new_ov = ov[['Date','TARGET']]
new_ov.head()

,Date,TARGET
1,2020-01-21,1
2,2020-01-22,1
3,2020-01-23,1
4,2020-01-24,2
5,2020-01-25,2


In [89]:
df = pd.merge(df, new_ov, on='Date')

## news 데이터

In [90]:
news = pd.read_csv("./data/corona_news.csv")

In [91]:
news.rename(columns={'date':'Date'},inplace=True)
news['Date'] = pd.to_datetime(news['Date'])
news.head()

,Date,counts
0,2020-01-20,256.0
1,2020-01-21,311.0
2,2020-01-22,430.0
3,2020-01-23,452.0
4,2020-01-24,421.0


In [92]:
df = pd.merge(df, news, on='Date')

In [93]:
df.rename(columns={'counts':'NEWS'},inplace=True)

# 구글 트렌드 데이터

In [94]:
trend = pd.read_csv("./data/구글트렌드.csv")

In [95]:
trend=trend[['Date', 'covid_tr', 'coro_tr']]

In [96]:
trend.Date = pd.to_datetime(trend.Date)

In [97]:
df = pd.merge(df, trend, on='Date')

In [98]:
df.covid_tr.iloc[-1] = 30
df.coro_tr.iloc[-1] = 17

In [99]:
# 데이터 저장
df.to_csv('./data/total_0509.csv')

In [100]:
df.tail()

,Date,EURO_conf,CHINA_conf,ASIA_conf,N_AMERICA_conf,NZ_conf,EURO_roam,CHINA_roam,ASIA_roam,N_AMERICA_roam,NZ_roam,TARGET,NEWS,covid_tr,coro_tr
100,2020-05-01,930930,83959,109157,1159804,1485,24.0,230.0,247.0,126.0,0.0,1073,1104.0,48.0,18.0
101,2020-05-02,942895,83959,114163,1190465,1487,77.0,90.0,254.0,174.0,0.0,1081,732.0,43.0,18.0
102,2020-05-03,950614,83964,119579,1218544,1487,48.0,130.0,248.0,104.0,0.0,1091,821.0,55.0,18.0
103,2020-05-04,957516,83966,125898,1242332,1486,51.0,94.0,238.0,95.0,0.0,1099,1331.0,30.0,17.0
104,2020-05-05,966279,83968,131413,1267566,1488,18.0,142.0,278.0,129.0,0.0,1102,793.0,30.0,17.0
